## Tutorial #5: Applying Machine Learning Methods to EEG Data on Group Level

In this tutorial, same classification tasks in Tutorial-4 will be examined but this time analysis will be done on group level.

##### Dataset: 
The the previous tutorial data of the only one participant in 'Emotion-Antecedent Appraisal Checks: EEG and EMG data sets for Novelty and Pleasantness' is used. In this tutorial, all participans will be included

In [2]:
import mne
from os.path import isfile, join
from os import listdir
import numpy as np

from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

# Models
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [3]:
#Load Dataset
data_folder = '../../study1/study1_eeg/epochdata/'
files = [data_folder+f for f in listdir(data_folder) if isfile(join(data_folder, f))]
#print(files)

# Read the EEG epochs:
epochs_all_UN, epochs_all_UP, epochs_all_NP = [], [], []
for f in files:
    epochs = mne.read_epochs(f, verbose=False)
    epochs_UN = epochs['FU', 'FN']
    epochs_UP = epochs['FU', 'FP']
    epochs_NP = epochs['FN', 'FP']
    epochs_all_UN.append(epochs_UN)
    epochs_all_UP.append(epochs_UP)
    epochs_all_NP.append(epochs_NP)
    
epochs_all_UN = np.array(epochs_all_UN)
epochs_all_UP = np.array(epochs_all_UP)
epochs_all_NP = np.array(epochs_all_NP)
print(epochs_all_UN.shape)
print(epochs_all_UP.shape)
print(epochs_all_NP.shape)

<ipython-input-3-dee499c60fbe>:9: RuntimeWarning: This filename (../../study1/study1_eeg/epochdata/P-15.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(f, verbose=False)
<ipython-input-3-dee499c60fbe>:9: RuntimeWarning: This filename (../../study1/study1_eeg/epochdata/P-01.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(f, verbose=False)
<ipython-input-3-dee499c60fbe>:9: RuntimeWarning: This filename (../../study1/study1_eeg/epochdata/P-14.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(f, verbose=False)
<ipython-input-3-dee499c60fbe>:9: RuntimeWarning: This filename (../../study1/study1_eeg/epochdata/P-02.fif) does not conform to MNE naming conventions. All epochs fi

(25,)
(25,)
(25,)


### Example #1:  Classification between Unpleasant and Neutral Events

In [4]:
# Dataset with unpleasant and neutral events
data_UN, labels_UN = [], []
for p in epochs_all_UN:
    data_UN.extend(p.get_data())
    labels_UN.extend(p.events[:,-1])
data_UN = np.array(data_UN)
labels_UN = np.array(labels_UN)
print(data_UN.shape)
print(labels_UN.shape)

(8575, 64, 384)
(8575,)


In [5]:
train_data_UN, test_data_UN, labels_train_UN, labels_test_UN = train_test_split(data_UN, labels_UN, test_size=0.3, random_state=42)

In [6]:
print(train_data_UN.shape)
print(test_data_UN.shape)

(6002, 64, 384)
(2573, 64, 384)


In [12]:
#svm
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), svm.SVC())
parameters = {'svc__kernel':['linear','rbf', 'sigmoid'], 'svc__C':[0.1, 1, 10]}
gs_cv_svm = GridSearchCV(clf_svm_pip, parameters, scoring='accuracy', cv=StratifiedKFold(n_splits=3), return_train_score=True)

In [ ]:
gs_cv_svm.fit(train_data_UN, labels_train_UN)
print('Best Parameters: {}'.format(gs_cv_svm.best_params_))
print('Best Score: {}'.format(gs_cv_svm.best_score_))

In [ ]:
predictions_svm = gs_cv_svm.predict(test_data_UN)
report_svm = classification_report(labels_test_UN, predictions_svm, target_names=['Unpleasant', 'Neutral'])
print('SVM Clasification Report:\n {}'.format(report_svm))
acc_svm = accuracy_score(labels_test_UN, predictions_svm)
print("Accuracy of SVM model: {}".format(acc_svm))